This notebook contains the code in data scraping.

In [1]:
# Import the dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import json
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import gkey

In [2]:
# Setting the url
url = "https://dojos.info/NewJersey/Counties.aspx"

# Creating the response
response = requests.get(url)

# Using beautiful soup
soup = bs(response.text,"html.parser")

# Code to create a list of counties.

county_tag = []
countylist = []

state = soup.find('ul', class_ = "olist")
county = state.find_all("a")
for i in range(len(county)):
    county_tag.append(county[i].text)

for j in county_tag:
    counties = j.replace(" ","")
    county_url = f"https://dojos.info/NewJersey/{counties}"
    countylist.append(county_url)
countylist

['https://dojos.info/NewJersey/AtlanticCounty',
 'https://dojos.info/NewJersey/BergenCounty',
 'https://dojos.info/NewJersey/BurlingtonCounty',
 'https://dojos.info/NewJersey/CamdenCounty',
 'https://dojos.info/NewJersey/CapeMayCounty',
 'https://dojos.info/NewJersey/CumberlandCounty',
 'https://dojos.info/NewJersey/EssexCounty',
 'https://dojos.info/NewJersey/GloucesterCounty',
 'https://dojos.info/NewJersey/HudsonCounty',
 'https://dojos.info/NewJersey/HunterdonCounty',
 'https://dojos.info/NewJersey/MercerCounty',
 'https://dojos.info/NewJersey/MiddlesexCounty',
 'https://dojos.info/NewJersey/MonmouthCounty',
 'https://dojos.info/NewJersey/MorrisCounty',
 'https://dojos.info/NewJersey/OceanCounty',
 'https://dojos.info/NewJersey/PassaicCounty',
 'https://dojos.info/NewJersey/SalemCounty',
 'https://dojos.info/NewJersey/SomersetCounty',
 'https://dojos.info/NewJersey/SussexCounty',
 'https://dojos.info/NewJersey/UnionCounty',
 'https://dojos.info/NewJersey/WarrenCounty']

In [3]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Kevin\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [ ]:
dojo_list = []

# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
time.sleep(2)


# Searches the website for the link to the counties
browser.links.find_by_partial_text("County").click()
time.sleep(1)

if browser.url == "https://dojos.info/NewJersey/Counties.aspx#google_vignette":
    browser.visit(url)
    browser.links.find_by_partial_text("County").click()

for i in range(1,len(countylist)):
    response = requests.get(browser.url)
    soup1 = bs(response.text,'html.parser')

    uList = soup1.find('ul',class_ = 'dojolist')
    dojolist = uList.find_all('h3')
    dojolen = len(dojolist)


    # Selects the dojo list to click.
    ulist = browser.find_by_css('.dojolist')

    h3 = ulist.find_by_tag('h3')

    # Click the the link to move to the dojo information.
    a_tags = h3.find_by_tag('a')

    a_tags.click()

    for x in range(1,dojolen + 1):

        # Finds the address for the dojo.
        address = browser.find_by_tag('address')

        # This will pull the address text
        p_address = address.text

        # This will split the text string into usable pieces.
        recreate = p_address.replace('\n'," | ").split("|")

        if len(recreate) > 3:
            recreate.pop(0)

        # This is the street address
        street = recreate[0].split()
        streets = " ".join(street)
        split = recreate[1].split(',')

        # The city address
        city = split[0]

        # Further splits the string into the state and zipcode parts.
        split2 = split[1].split(" ")
        state = split2[1]
        zipcode = split2[2]

        # This will pull the name of the dojo.
        dojo_name = browser.find_by_css('#pageTop')
        dojo = dojo_name.find_by_tag("h1").text

        # This will find the styles that the dojo teaches
        response = requests.get(browser.url)
        soup2 = bs(response.text , "html")

        style = []

        styles = soup2.find('ul', class_ = 'styles')
        for dank in styles.find_all('li'):
            style.append(dank.text)

        # Finding the phone number.
        phone = browser.find_by_css('.phone')
        dojo_phone = phone.text.split(" ")[1]

        dojo_dict = {'Name': dojo,
                    'Phone': dojo_phone,
                    'Zipcode': zipcode,
                    'Street': streets,
                    'City': city,
                    'State': state,
                    'Style': style}

        dojo_list.append(dojo_dict)
        browser.back()
        time.sleep(1)

        try:
            ulist = browser.find_by_css('.dojolist')
            h3 = ulist.find_by_tag('h3')[x]
            h3.find_by_tag("a").click()

        except:
            print('done')
    
    try:
        browser.visit(countylist[i])
    except:
        print('nice')

In [ ]:
# Quits out of the browser.
browser.quit()

In [ ]:
call_list = []
for rows in range(len(dojo_list)):
    strt = dojo_list[rows]['Street']
    city = dojo_list[rows]['City']
    state = dojo_list[rows]['State']
    key = 'dank'
    api_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={strt},{city},+{state}&key={gkey}'
    api_url
    call_list.append(api_url.replace(" ","+"))

In [ ]:
geo_data = requests.get(call_list[0]).json()
print(geo_data)

In [ ]:
print(json.dumps(geo_data, indent=4, sort_keys=True))

In [ ]:
df = pd.DataFrame(dojo_list)
df

In [ ]:
dojo_list

In [ ]:
# Selects the dojo list to click.
ulist = browser.find_by_css('.dojolist')

# Click the the link to move to the dojo information.
ulist.find_by_tag('a')

ulist

In [ ]:
# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
# Launches browser to search
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
dojo_list = []

# Visits the url
url = "https://dojos.info/NewJersey/Counties.aspx"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
time.sleep(2)

In [ ]:
url2 = county[0]
response = requests.get(url2)
soup = bs(response.text,"html.parser")
dank = []
for ulist in soup.find_all('ul', class_ = 'dojolist'):
    for h3 in ulist.find_all('h3'):
        for a in h3.find_all('a'):
            dank.append(a['href'])

In [ ]:
dank

In [ ]:
dojo = []
for k in dank:
    dojo_url = f"https://dojos.info{k}"
    dojo.append(dojo_url)

In [ ]:
url3 = dojo[0]
response = requests.get(url3)
soup = bs(response.text,"html.parser")

address = soup.find("address")
new_address = address.get_text("|").split("|")
street1 = new_address[0].replace("\r\n\t\t","")
street2 = street1.split()
street = " ".join(street2)
street


In [ ]:
new_address

In [ ]:
# Finds the address for the dojo.
address = browser.find_by_tag('address')

# This will pull the address text
p_address = address.text

# This will split the text string into usable pieces.
recreate = p_address.replace('\n'," | ").split("|")
street = recreate[0]
split = recreate[1].split(',')
city = split[0]

split2 = split[1].split(" ")
state = split2[1]
zipcode = split2[2]

# This will pull the name of the dojo.
dojo_name = browser.find_by_css('#pageTop')
dojo = dojo_name.find_by_tag("h1").text

# Finding the phone number.
phone = browser.find_by_css('.phone')
dojo_phone = phone.text.split(" ")[1]

In [ ]:
county

In [ ]:
county_url2 = 'https://dojos.info/NewJersey/AtlanticCounty/'
response = requests.get(county_url2)

soup = bs(response.text,"html.parser")
ulist = soup.find('ul', class_ ='dojolist')

dojo_link = ulist.find('h3')
dojo_link

In [ ]:
recreate

In [ ]:
response = requests.get(browser.url)
soup = bs(response.text , "html")

style = []

styles = soup.find('ul', class_ = 'styles')
for dank in styles.find_all('li'):
    style.append(dank.text)
style

The Code below is to start pulling information on the dojo

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

In [ ]:
address = soup.find('address')
address

In [ ]:
parts = address.find_all("a")

In [ ]:
for i in range(3):
    print(parts[i].text)

In [ ]:
part = soup.find('address', id='text')

In [ ]:
# Finding the phone number.
phone = browser.find_by_css('.phone')
dojo_phone = phone.text.split(" ")
dojo_phone[1]

In [ ]:
address.text

In [ ]:
address

Pulls the Address

In [ ]:
new_address = address.get_text("|")

In [ ]:
new_address.replace(" ","")

In [ ]:
first = new_address.split("|")[0]

In [ ]:
new_address.split("|")

In [ ]:
new_address.replace("\r\n\t\t","")

In [ ]:
second = first.replace("\r\n\t\t","")

In [ ]:
# Splits and joins the string.
third = second.split()
fourth = " ".join(third)
fourth

In [ ]:
dojo_name = soup.find("div", id ="pageTop")
dojo_name.find("h1").get_text()

In [ ]:
url2 = 'https://dojos.info/Acmfs/'
response = requests.get(url2)

# Using beautiful soup
soup = bs(response.text,"html.parser")

address = soup.find('address')
new_address = address.get_text("|")
clean_address = new_address.replace("\r\n\t\t","")
parts = clean_address.split("|")

In [ ]:
parts

In [ ]:
address.get_text("|")\
    .replace("\r\n\t\t","")\
    .split("|")